In [ ]:
import nltk
nltk.download('punkt')       
nltk.download('stopwords') 
nltk.download('punkt_tab')  
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
import numpy as np
import pandas as pd
from helper_functions import *
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from collections import Counter

In [ ]:
# Load in Recall data

# Paths to all three files
recall_files = [
    "../Data/Current Version of Toys Incidence+Recall/Toysandchildren_ArtsandCrafts.csv",
    "../Data/Current Version of Toys Incidence+Recall/Toysandchildren_Riding_Toys.csv",
    "../Data/Current Version of Toys Incidence+Recall/Toysandchildren_Toys.csv"
]

recall_dfs = [load_clean_csv(path) for path in recall_files]
recalls_df = pd.concat(recall_dfs, ignore_index=True)



In [ ]:
# pre process the text
recalls_df['lemmas'] = recalls_df['Incident Description'].apply(preprocess)

In [ ]:
!pip install yake

In [ ]:
import yake

In [ ]:
kw_extractor = yake.KeywordExtractor(lan="en", n=1, top=10)

In [ ]:
def extract_keywords_from_lemmas(lemmas_list, top_n=10):
    text = " ".join(lemmas_list)
    kw_tuples = kw_extractor.extract_keywords(text)
    # If you only want the keywords themselves:
    keywords = [kw for kw, score in kw_tuples][:top_n]
    return keywords

# Apply to each row, storing the resulting keyword list in a new column:
recalls_df["keywords"] = recalls_df["lemmas"].apply(lambda lem: extract_keywords_from_lemmas(lem))

recalls_df

In [ ]:
recalls_df['keywords'].to_clipboard()